## Utility Functions

In [1]:
directives = ['.BSC', '.BSS', '.END', '.ORG'] # '.EQU'
insts = {
# I-type
    'BNE': ['I', 0, 's', 't', 'l'],
    'BEQ': ['I', 1, 's', 't', 'l'],
    'BGZ': ['I', 2, 's', 'l'],
    'BLZ': ['I', 3, 's', 'l'],
    'ADI': ['I', 4, 't', 's', 'i8'],
    'ORI': ['I', 5, 't', 's', 'u8'],
    'LHI': ['I', 6, 't', 'u8'],
    'LWD': ['I', 7, 't', 's', 'i8'],
    'SWD': ['I', 8, 't', 's', 'i8'],

# J-type
    'JMP': ['J', 9, 'j'],
    'JAL': ['J', 10, 'j'],

# R-type
    'ADD': ['R', 0, 'd', 's', 't'],
    'SUB': ['R', 1, 'd', 's', 't'],
    'AND': ['R', 2, 'd', 's', 't'],
    'ORR': ['R', 3, 'd', 's', 't'],
    'NOT': ['R', 4, 'd', 's'],
    'TCP': ['R', 5, 'd', 's'],
    'SHL': ['R', 6, 'd', 's'],
    'SHR': ['R', 7, 'd', 's'],
    'JPR': ['R', 25, 's'],
    'JRL': ['R', 26, 's'],
    'RWD': ['R', 27, 'd'],
    'WWD': ['R', 28, 's'],
    'HLT': ['R', 29],
    'ENI': ['R', 30, 's'],
    'DSI': ['R', 31, 's'],
}

def check_name(name: str):
    if len(name) <= 0:
        return False
    elif name[0].isdigit():
        return False
    elif name in directives:
        return False
    else:
        return True
    
def parse_int(s: str):
    if len(s) <= 0:
        raise ValueError("parse_int(): Empty String!")
    # support x86 hexadecimal
    if s[-1] == 'h':
        return int(s[:-1], base=16)
    # literal
    if s[0] == '0':
        if len(s) == 1:
            return 0
        elif s[1] == 'b':
            return int(s[2:], base=2)
        elif s[1] == 'o':
            return int(s[2:], base=8)
        elif s[1] == 'x':
            return int(s[2:], base=16)
        elif s[1].isdigit():
            return int(s[1:], base=8)
        else:
            raise ValueError("parse_int(): Unsupported Literal!")
    elif (s[0] in ['+', '-']) and s[1] == '0':
        if len(s) == 2:
            return 0
        elif s[2] == 'b':
            return int(s[0]+s[3:], base=2)
        elif s[2] == 'o':
            return int(s[0]+s[3:], base=8)
        elif s[2] == 'x':
            return int(s[0]+s[3:], base=16)
        elif s[2].isdigit():
            return int(s[0]+s[2:], base=8)
        else:
            raise ValueError("parse_int(): Unsupported Literal!")
    # decimal
    return int(s)

## Assembly Code (`CPU_TB.v`)

In [2]:
asm = """
.ORG    0
    JMP    ENTRY
VAR1    .BSC 0x0001
VAR2    .BSC 0xFFFF
STACK   .BSS 32
ENTRY:
    LHI     $0, 0
    WWD     $0          ; TEST #1-1 : LHI (= 0x0000)
    LHI     $1, 0
    WWD     $1          ; TEST #1-2 : LHI (= 0x0000)
    LHI     $2, 0
    WWD     $2          ; TEST #1-3 : LHI (= 0x0000)
    LHI     $3, 0
    WWD     $3          ; TEST #1-4 : LHI (= 0x0000)
    ADI     $0, $1, 1
    WWD     $0          ; TEST #2-1 : ADI (= 0x0001)
    ADI     $0, $0, 1
    WWD     $0          ; TEST #2-2 : ADI (= 0x0002)
    ORI     $1, $2, 1
    WWD     $1          ; TEST #3-1 : ORI (= 0x0001)
    ORI     $1, $1, 2
    WWD     $1          ; TEST #3-2 : ORI (= 0x0003)
    ORI     $1, $1, 3
    WWD     $1          ; TEST #3-3 : ORI (= 0x0003)
    ADD     $3, $0, $2
    WWD     $3          ; TEST #4-1 : ADD (= 0x0002)
    ADD     $3, $1, $2
    WWD     $3          ; TEST #4-2 : ADD (= 0x0003)
    ADD     $3, $0, $1
    WWD     $3          ; TEST #4-3 : ADD (= 0x0005)
    SUB     $3, $0, $2
    WWD     $3          ; TEST #5-1 : SUB (= 0x0002)
    SUB     $3, $2, $0
    WWD     $3          ; TEST #5-2 : SUB (= 0xFFFE)
    SUB     $3, $1, $2
    WWD     $3          ; TEST #5-3 : SUB (= 0x0003)
    SUB     $3, $2, $1
    WWD     $3          ; TEST #5-4 : SUB (= 0xFFFD)
    SUB     $3, $0, $1
    WWD     $3          ; TEST #5-5 : SUB (= 0xFFFF)
    SUB     $3, $1, $0
    WWD     $3          ; TEST #5-6 : SUB (= 0x0001)
    AND     $3, $0, $2
    WWD     $3          ; TEST #6-1 : AND (= 0x0000)
    AND     $3, $1, $2
    WWD     $3          ; TEST #6-2 : AND (= 0x0000)
    AND     $3, $0, $1
    WWD     $3          ; TEST #6-3 : AND (= 0x0002)
    ORR     $3, $0, $2
    WWD     $3          ; TEST #7-1 : ORR (= 0x0002)
    ORR     $3, $1, $2
    WWD     $3          ; TEST #7-2 : ORR (= 0x0003)
    ORR     $3, $0, $1
    WWD     $3          ; TEST #7-3 : ORR (= 0x0003)
    NOT     $3, $0
    WWD     $3          ; TEST #8-1 : NOT (= 0xFFFD)
    NOT     $3, $1
    WWD     $3          ; TEST #8-2 : NOT (= 0xFFFC)
    NOT     $3, $2
    WWD     $3          ; TEST #8-3 : NOT (= 0xFFFF)
    TCP     $3, $0
    WWD     $3          ; TEST #9-1 : TCP (= 0xFFFE)
    TCP     $3, $1
    WWD     $3          ; TEST #9-2 : TCP (= 0xFFFD)
    TCP     $3, $2
    WWD     $3          ; TEST #9-3 : TCP (= 0x0000)
    SHL     $3, $0
    WWD     $3          ; TEST #10-1 : SHL (= 0x0004)
    SHL     $3, $1
    WWD     $3          ; TEST #10-2 : SHL (= 0x0006)
    SHL     $3, $2
    WWD     $3          ; TEST #10-3 : SHL (= 0x0000)
    SHR     $3, $0
    WWD     $3          ; TEST #11-1 : SHR (= 0x0001)
    SHR     $3, $1
    WWD     $3          ; TEST #11-2 : SHR (= 0x0001)
    SHR     $3, $2
    WWD     $3          ; TEST #11-3 : SHR (= 0x0000)
    LWD     $0, $2, VAR1
    WWD     $0          ; TEST #12-1 : LWD (= 0x0001)
    LWD     $1, $2, VAR2
    WWD     $1          ; TEST #12-2 : LWD (= 0xFFFF)
    SWD     $1, $2, VAR1
    SWD     $0, $2, VAR2
    LWD     $0, $2, VAR1
    WWD     $0          ; TEST #13-1 : WWD (= 0xFFFF)
    LWD     $1, $2, VAR2
    WWD     $1          ; TEST #13-2 : WWD (= 0x0001)
    JMP     JMP0
JMP0:
    WWD     $0          ; TEST #14-1 : JMP (= 0xFFFF)
    JMP     JMP1
    HLT
JMP1:
    WWD     $1          ; TEST #14-2 : JMP (= 0x0001)
    BNE     $2, $3, BNE1
    JMP     BNE2
BNE1:
    HLT
BNE2:
    WWD     $0          ; TEST #15-1 : BNE (= 0xFFFF)
    BNE     $1, $2, BNE3
    HLT
BNE3:
    WWD     $1          ; TEST #15-2 : BNE (= 0x0001)
    BEQ     $1, $2, BEQ1
    JMP     BEQ2
BEQ1:
    HLT
BEQ2:
    WWD     $0          ; TEST #16-1 : BEQ (= 0xFFFF)
    BEQ     $2, $3, BEQ3
    HLT
BEQ3:
    WWD     $1          ; TEST #16-2 : BEQ (= 0x0001)
    BGZ     $0, BGZ1
    JMP     BGZ2
BGZ1:
    HLT
BGZ2:
    WWD     $0          ; TEST #17-1 : BGZ (= 0xFFFF)
    BGZ     $1, BGZ3
    HLT
BGZ3:
    WWD     $1          ; TEST #17-2 : BGZ (= 0x0001)
    BGZ     $2, BGZ4
    JMP     BGZ5
BGZ4:
    HLT
BGZ5:
    WWD     $0          ; TEST #17-3 : BGZ (= 0xFFFF)
    BLZ     $0, BLZ1
    HLT
BLZ1:
    WWD     $1          ; TEST #18-1 : BLZ (= 0x0001)
    BLZ     $1, BLZ2
    JMP     BLZ3
BLZ2:
    HLT
BLZ3:
    WWD     $0          ; TEST #18-2 : BLZ (= 0xFFFF)
    BLZ     $2, BLZ4
    JMP     BLZ5
BLZ4:
    HLT
BLZ5:
    WWD     $1          ; TEST #18-3 : BLZ (= 0x0001)
    JAL     SIMPLE1
    WWD     $0          ; TEST #19-1 : JAL & JPR (= 0xFFFF)
    JAL     SIMPLE2
    HLT
    WWD     $1          ; TEST #19-2 : JAL & JPR (= 0x0001)
    LHI     $3, 0
    ORI     $3, $3, STACK
    LHI     $0, 0
    ADI     $0, $0, 5
    JAL     FIB
    WWD     $0          ; TEST #19-3 : JAL & JPR (= 0x0008)
    JMP     PREFIB1
PREFIB2:
    ADI     $1, $2, 0
    JRL     $1
    WWD     $0          ; TEST #20 : JAL & JRL & JPR (= 0x0022)
    JMP     PRELOOP
SIMPLE2:
    ADI     $2, $2, 1
SIMPLE1:
    JPR     $2
    HLT
PREFIB1:
    JAL     PREFIB2
FIB:
    ADI     $1, $0, -1
    BGZ     $1, FIBRECUR
    LHI     $0, 0
    ORI     $0, $0, 1
    JPR     $2
    HLT
FIBRECUR:
    SWD     $2, $3, 0
    SWD     $0, $3, 1
    ADI     $3, $3, 2
    ADI     $0, $0, -2
    JAL     FIB
    LWD     $1, $3, -1
    SWD     $0, $3, -1
    ADI     $0, $1, -1
    JAL     FIB
    LWD     $1, $3, -1
    LWD     $2, $3, -2
    ADD     $0, $0, $1
    ADI     $3, $3, -2
    JPR     $2
    HLT
PRELOOP:
    LHI     $3, 1
    LHI     $0, 0
    ADI     $2, $1, 16
LOOP:
    LWD     $0, $2, 0
    ADD     $1, $1, $0
    ADI     $2, $2, 1
    LHI     $0, 0
    ADI     $0, $0, 23
    AND     $2, $2, $0
    ADI     $3, $3, -1
    BGZ     $3, LOOP
    WWD     $1          ; TEST #21-1 : ADI & BGZ (= 0x002D)
    LHI     $0, 0
    WWD     $2          ; TEST #21-2 : ADI & BGZ (= 0x0015)
    ADI     $1, $0, 24
    ADI     $0, $0, 8
    LHI     $2, 0
SETZERO:
    SWD     $2, $1, 0
    ADI     $1, $1, 1
    ADI     $0, $0, -1
    BGZ     $0, SETZERO
    ORI     $3, $2, STACK
    ADI     $0, $2, 5
    ADI     $1, $2, 24
    JAL     FIBDP
    WWD     $0          ; TEST #22 : JAL & SWD & LWD & JPR (= 0x0008)
    HLT                 ; FINISHED
FIBDP:
    SWD     $2, $3, 0
    SWD     $1, $3, 1
    SWD     $0, $3, 2
    ADI     $3, $3, 4
    ADI     $2, $0, -2
    ADD     $1, $1, $0
    LWD     $0, $1, 0
    BGZ     $0, DPEND
    LHI     $0, 0
    ADI     $0, $0, 1
    BLZ     $2, SETDP
    ADI     $0, $2, 1
    LWD     $1, $3, -3
    JAL     FIBDP
    SWD     $0, $3, -1
    LWD     $0, $3, -2
    LWD     $1, $3, -3
    ADI     $0, $0, -2
    JAL     FIBDP
    LWD     $1, $3, -1
    ADD     $0, $0, $1
SETDP:
    LWD     $2, $3, -2
    LWD     $1, $3, -3
    ADD     $1, $1, $2
    SWD     $0, $1, 0
DPEND:
    LWD     $2, $3, -4
    ADI     $3, $3, -4
    JPR     $2
    HLT
TESTEND:
    HLT                 ; FINISHED
.END
"""

## Parser

In [3]:
tokens = []
n_lines = 0
n_token = 0

for line in asm.splitlines():
    # remove comments
    code = line.strip().split(';')[0].split()
    if len(code) <= 0:
        pass
    else:
        cstr = ' '.join(code)
        n_lines += 1
        # Label
        if ':' in cstr:
            i = cstr.index(':')
            if not check_name(cstr[:i]):
                print("LABEL NAME ERROR!")
                break
            else:
                __code = ('l', cstr[:i])
                tokens.append(__code)
                n_token += 1
                if (i+1) >= len(cstr):
                    continue
                cstr = cstr[i+1:].strip()
                code = cstr.split()
        # Directive
        if '.' in cstr:
            try:
                if code[0][0] == '.':
                    dir = code[0]
                    __code = ('d', dir, *[i.strip() for i in ' '.join(code[1:]).split(',')])
                elif code[1][0] == '.':
                    dir = code[1]
                    __code = ('D', dir, *[i.strip() for i in ' '.join(code[2:]).split(',')], code[0])
                tokens.append(__code)
                n_token += 1
                continue
            except:
                if code[0] == '.END':
                    __code = ('d', '.END')
                    tokens.append(__code)
                    n_token += 1
                    continue
                pass
        # Inst str
        if len(code) == 1:
            __code = ('i', code[0])
        else:
            __code = ('i', code[0], *[i.strip() for i in ' '.join(code[1:]).split(',')])
        tokens.append(__code)
        n_token += 1

print(len(tokens), f"## {n_lines} non-empty lines: {n_token} tokens")

259 ## 259 non-empty lines: 259 tokens


In [4]:
for token in tokens:
    print(token)

('d', '.ORG', '0')
('i', 'JMP', 'ENTRY')
('D', '.BSC', '0x0001', 'VAR1')
('D', '.BSC', '0xFFFF', 'VAR2')
('D', '.BSS', '32', 'STACK')
('l', 'ENTRY')
('i', 'LHI', '$0', '0')
('i', 'WWD', '$0')
('i', 'LHI', '$1', '0')
('i', 'WWD', '$1')
('i', 'LHI', '$2', '0')
('i', 'WWD', '$2')
('i', 'LHI', '$3', '0')
('i', 'WWD', '$3')
('i', 'ADI', '$0', '$1', '1')
('i', 'WWD', '$0')
('i', 'ADI', '$0', '$0', '1')
('i', 'WWD', '$0')
('i', 'ORI', '$1', '$2', '1')
('i', 'WWD', '$1')
('i', 'ORI', '$1', '$1', '2')
('i', 'WWD', '$1')
('i', 'ORI', '$1', '$1', '3')
('i', 'WWD', '$1')
('i', 'ADD', '$3', '$0', '$2')
('i', 'WWD', '$3')
('i', 'ADD', '$3', '$1', '$2')
('i', 'WWD', '$3')
('i', 'ADD', '$3', '$0', '$1')
('i', 'WWD', '$3')
('i', 'SUB', '$3', '$0', '$2')
('i', 'WWD', '$3')
('i', 'SUB', '$3', '$2', '$0')
('i', 'WWD', '$3')
('i', 'SUB', '$3', '$1', '$2')
('i', 'WWD', '$3')
('i', 'SUB', '$3', '$2', '$1')
('i', 'WWD', '$3')
('i', 'SUB', '$3', '$0', '$1')
('i', 'WWD', '$3')
('i', 'SUB', '$3', '$1', '$0')
('i

## Pass-1&2

In [5]:
# Pass 1
addr = 0; labels = dict()

for token in tokens:
    match token[0]:
        case 'd':
            match token[1]:
                case '.BSC':
                    addr += len(token) - 2
                case '.BSS':
                    if (sz := parse_int(token[2])) > 0:
                        addr += sz
                case '.END':
                    break
                case '.ORG':
                    if (org := parse_int(token[2])) >= addr:
                        addr = org
                    else:
                        print("[Pass 1] Warning: section start is before than previous section data end")
                        addr = org
                case _:
                    pass
        case 'D':
            labels[token[-1]] = addr
            match token[1]:
                case '.BSC':
                    addr += len(token) - 3
                case '.BSS':
                    if (sz := parse_int(token[2])) > 0:
                        addr += sz
                case '.END':
                    break
                case '.ORG':
                    if (org := parse_int(token[2])) >= addr:
                        addr = org
                    else:
                        print("[Pass 1] Warning: section start is before than previous section data end")
                        addr = org
                case _:
                    pass
        case 'l':
            labels[token[1]] = addr
        case 'i':
            addr += 1
        case _:
            pass

print("[Pass 1]")
print("Resolved Symbols:", labels)

# Pass 2
addr = 0; binary = []

for token in tokens:
    match token[0]:
        case 'd':
            match token[1]:
                case '.BSC':
                    for word in token[2:]:
                        try:
                            binary.append((addr, (parse_int(word) & 0xffff)))
                            addr += 1
                        except:
                            try:
                                if word in labels.keys():
                                    binary.append((addr, (labels[word] & 0xffff)))
                                    addr += 1
                            except:
                                raise ValueError("[Pass 2] Error: Invalid Label Name!")
                case '.BSS':
                    if (sz := parse_int(token[2])) > 0:
                        for _ in range(sz):
                            binary.append((addr, 0))
                            addr += 1
                case '.END':
                    break
                case '.ORG':
                    addr = org
                case _:
                    pass
        case 'D':
            match token[1]:
                case '.BSC':
                    for word in token[2:-1]:
                        try:
                            binary.append((addr, (parse_int(word) & 0xffff)))
                            addr += 1
                        except:
                            try:
                                if word in labels.keys():
                                    binary.append((addr, (labels[word] & 0xffff)))
                                    addr += 1
                            except:
                                raise ValueError("[Pass 2] Error: Invalid Label Name!")
                case '.BSS':
                    if (sz := parse_int(token[2])) > 0:
                        for _ in range(sz):
                            binary.append((addr, 0))
                            addr += 1
                case '.END':
                    break
                case '.ORG':
                    addr = org
                case _:
                    pass
        case 'l':
            pass
        case 'i':
            op = insts[token[1]]
            word = 0x0000
            match op[0]:
                case 'I':
                    word |= op[1] << 12
                    for i, operand in enumerate(op):
                        if i < 2:
                            pass
                        match operand:
                            case 's':
                                if token[i][0] == '$':
                                    regno = int(token[i][1:])
                                    if regno not in range(4):
                                        raise ValueError("[Pass 2] Error: Invalid Register #!")
                                    word |= regno << 10
                            case 't':
                                if token[i][0] == '$':
                                    regno = int(token[i][1:])
                                    if regno not in range(4):
                                        raise ValueError("[Pass 2] Error: Invalid Register #!")
                                    word |= regno << 8
                            case 'i8':
                                try:
                                    imm = parse_int(token[i])
                                except:
                                    try:
                                        if token[i] in labels.keys():
                                            imm = labels[token[i]]
                                    except:
                                        raise ValueError("[Pass 2] Error: Invalid Immediate or Offset!")
                                if imm not in range(-128, 128):
                                    raise ValueError("[Pass 2] Error: Immediate Out of Range!")
                                word |= imm & 0xff
                            case 'u8':
                                try:
                                    imm = parse_int(token[i])
                                except:
                                    try:
                                        if token[i] in labels.keys():
                                            imm = labels[token[i]]
                                    except:
                                        raise ValueError("[Pass 2] Error: Invalid Immediate or Offset!")
                                if imm not in range(0, 256):
                                    raise ValueError("[Pass 2] Error: Immediate Out of Range!")
                                word |= imm & 0xff
                            case 'l':
                                try:
                                    imm = parse_int(token[i])
                                except:
                                    try:
                                        if token[i] in labels.keys():
                                            imm = labels[token[i]] - (addr + 1)
                                    except:
                                        raise ValueError("[Pass 2] Error: Invalid Offset!")
                                if imm not in range(-128, 128):
                                    raise ValueError("[Pass 2] Error: Immediate Out of Range!")
                                word |= imm & 0xff
                            case _:
                                pass
                case 'J':
                    word |= op[1] << 12
                    try:
                        target = parse_int(token[2])
                        if (target ^ addr) & 0xf000 != 0:
                            raise ValueError("[Pass 2] Error: Invalid Jump Target!")
                        offset = target & 0x0fff
                    except:
                        try:
                            if token[2] in labels.keys():
                                target = labels[token[2]]
                                if (target ^ addr) & 0xf000 != 0:
                                    raise ValueError("[Pass 2] Error: Invalid Jump Target!")
                                offset = target & 0x0fff
                        except:
                            raise ValueError("[Pass 2] Error: Invalid Immediate or Offset!")
                    word |= offset
                case 'R':
                    word |= 15 << 12
                    word |= op[1]
                    for i, operand in enumerate(op):
                        if i < 2:
                            pass
                        match operand:
                            case 's':
                                if token[i][0] == '$':
                                    regno = int(token[i][1:])
                                    if regno not in range(4):
                                        raise ValueError("[Pass 2] Error: Invalid Register #!")
                                    word |= regno << 10
                            case 't':
                                if token[i][0] == '$':
                                    regno = int(token[i][1:])
                                    if regno not in range(4):
                                        raise ValueError("[Pass 2] Error: Invalid Register #!")
                                    word |= regno << 8
                            case 'd':
                                if token[i][0] == '$':
                                    regno = int(token[i][1:])
                                    if regno not in range(4):
                                        raise ValueError("[Pass 2] Error: Invalid Register #!")
                                    word |= regno << 6
                            case _:
                                pass
                case _:
                    pass
            binary.append((addr, (word & 0xffff)))
            addr += 1
        case _:
            pass

print("[Pass 2]")
print(f"Finished at 0x{addr:x}")

[Pass 1]
Resolved Symbols: {'VAR1': 1, 'VAR2': 2, 'STACK': 3, 'ENTRY': 35, 'JMP0': 118, 'JMP1': 121, 'BNE1': 124, 'BNE2': 125, 'BNE3': 128, 'BEQ1': 131, 'BEQ2': 132, 'BEQ3': 135, 'BGZ1': 138, 'BGZ2': 139, 'BGZ3': 142, 'BGZ4': 145, 'BGZ5': 146, 'BLZ1': 149, 'BLZ2': 152, 'BLZ3': 153, 'BLZ4': 156, 'BLZ5': 157, 'PREFIB2': 170, 'SIMPLE2': 174, 'SIMPLE1': 175, 'PREFIB1': 177, 'FIB': 178, 'FIBRECUR': 184, 'PRELOOP': 199, 'LOOP': 202, 'SETZERO': 216, 'FIBDP': 226, 'SETDP': 247, 'DPEND': 251, 'TESTEND': 255}
[Pass 2]
Finished at 0x100


In [6]:
print(len(binary))

256


In [7]:
for addr, word in binary:
    print(f"{addr:04x}: {word:04x}")

0000: 9023
0001: 0001
0002: ffff
0003: 0000
0004: 0000
0005: 0000
0006: 0000
0007: 0000
0008: 0000
0009: 0000
000a: 0000
000b: 0000
000c: 0000
000d: 0000
000e: 0000
000f: 0000
0010: 0000
0011: 0000
0012: 0000
0013: 0000
0014: 0000
0015: 0000
0016: 0000
0017: 0000
0018: 0000
0019: 0000
001a: 0000
001b: 0000
001c: 0000
001d: 0000
001e: 0000
001f: 0000
0020: 0000
0021: 0000
0022: 0000
0023: 6000
0024: f01c
0025: 6100
0026: f41c
0027: 6200
0028: f81c
0029: 6300
002a: fc1c
002b: 4401
002c: f01c
002d: 4001
002e: f01c
002f: 5901
0030: f41c
0031: 5502
0032: f41c
0033: 5503
0034: f41c
0035: f2c0
0036: fc1c
0037: f6c0
0038: fc1c
0039: f1c0
003a: fc1c
003b: f2c1
003c: fc1c
003d: f8c1
003e: fc1c
003f: f6c1
0040: fc1c
0041: f9c1
0042: fc1c
0043: f1c1
0044: fc1c
0045: f4c1
0046: fc1c
0047: f2c2
0048: fc1c
0049: f6c2
004a: fc1c
004b: f1c2
004c: fc1c
004d: f2c3
004e: fc1c
004f: f6c3
0050: fc1c
0051: f1c3
0052: fc1c
0053: f0c4
0054: fc1c
0055: f4c4
0056: fc1c
0057: f8c4
0058: fc1c
0059: f0c5
005a: fc1c

In [8]:
for addr, word in binary:
    print(f"				memory[16'h{addr:x}] <= 16'h{word:x};")

				memory[16'h0] <= 16'h9023;
				memory[16'h1] <= 16'h1;
				memory[16'h2] <= 16'hffff;
				memory[16'h3] <= 16'h0;
				memory[16'h4] <= 16'h0;
				memory[16'h5] <= 16'h0;
				memory[16'h6] <= 16'h0;
				memory[16'h7] <= 16'h0;
				memory[16'h8] <= 16'h0;
				memory[16'h9] <= 16'h0;
				memory[16'ha] <= 16'h0;
				memory[16'hb] <= 16'h0;
				memory[16'hc] <= 16'h0;
				memory[16'hd] <= 16'h0;
				memory[16'he] <= 16'h0;
				memory[16'hf] <= 16'h0;
				memory[16'h10] <= 16'h0;
				memory[16'h11] <= 16'h0;
				memory[16'h12] <= 16'h0;
				memory[16'h13] <= 16'h0;
				memory[16'h14] <= 16'h0;
				memory[16'h15] <= 16'h0;
				memory[16'h16] <= 16'h0;
				memory[16'h17] <= 16'h0;
				memory[16'h18] <= 16'h0;
				memory[16'h19] <= 16'h0;
				memory[16'h1a] <= 16'h0;
				memory[16'h1b] <= 16'h0;
				memory[16'h1c] <= 16'h0;
				memory[16'h1d] <= 16'h0;
				memory[16'h1e] <= 16'h0;
				memory[16'h1f] <= 16'h0;
				memory[16'h20] <= 16'h0;
				memory[16'h21] <= 16'h0;
				memory[16'h22] <= 16

In [9]:
print('[', end='')
for i, d in enumerate(binary):
    addr, data = d
    print(f"(0x{addr:x}, 0x{data:x}), ", end='')
    if (i & 0x3) == 0x3:
        print()
print(']')

[(0x0, 0x9023), (0x1, 0x1), (0x2, 0xffff), (0x3, 0x0), 
(0x4, 0x0), (0x5, 0x0), (0x6, 0x0), (0x7, 0x0), 
(0x8, 0x0), (0x9, 0x0), (0xa, 0x0), (0xb, 0x0), 
(0xc, 0x0), (0xd, 0x0), (0xe, 0x0), (0xf, 0x0), 
(0x10, 0x0), (0x11, 0x0), (0x12, 0x0), (0x13, 0x0), 
(0x14, 0x0), (0x15, 0x0), (0x16, 0x0), (0x17, 0x0), 
(0x18, 0x0), (0x19, 0x0), (0x1a, 0x0), (0x1b, 0x0), 
(0x1c, 0x0), (0x1d, 0x0), (0x1e, 0x0), (0x1f, 0x0), 
(0x20, 0x0), (0x21, 0x0), (0x22, 0x0), (0x23, 0x6000), 
(0x24, 0xf01c), (0x25, 0x6100), (0x26, 0xf41c), (0x27, 0x6200), 
(0x28, 0xf81c), (0x29, 0x6300), (0x2a, 0xfc1c), (0x2b, 0x4401), 
(0x2c, 0xf01c), (0x2d, 0x4001), (0x2e, 0xf01c), (0x2f, 0x5901), 
(0x30, 0xf41c), (0x31, 0x5502), (0x32, 0xf41c), (0x33, 0x5503), 
(0x34, 0xf41c), (0x35, 0xf2c0), (0x36, 0xfc1c), (0x37, 0xf6c0), 
(0x38, 0xfc1c), (0x39, 0xf1c0), (0x3a, 0xfc1c), (0x3b, 0xf2c1), 
(0x3c, 0xfc1c), (0x3d, 0xf8c1), (0x3e, 0xfc1c), (0x3f, 0xf6c1), 
(0x40, 0xfc1c), (0x41, 0xf9c1), (0x42, 0xfc1c), (0x43, 0xf1c1), 
(0x44, 0x